# Database Querying using SQLite

## Understand the datasets

<p>You will be using three datasets that are available on the city of Chicago's Data Portal to work out the problems.</p>
<ol>
    <li><a href="https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2/about_data">Socioeconomic Indicators in Chicago</a>: This dataset contains a selection of six socioeconomic indicators of public health significance and a "hardship index", for each Chicago community area, for the years 2008 – 2012.</li>
    <li><a href="https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t/about_data">Chicago Public Schools</a>: This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.</li>
    <li><a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/about_data">Chicago Crime Data</a>: This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.</li>
</ol>

## Download the datasets

<p>In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the Internet.</p>
<p>Use the links below to read the data files using the Pandas library.</p>
<ul>
    <li>Chicago Census Data - Selected socioeconomic indicators in Chicago: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv">Link 1</a></li>
    <li>Chicago Public Schools: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv">Link 2</a></li>
    <li>Chicago Crime Data: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv">Link 3</a></li>
</ul>
<p><b>NOTE:</b> Ensure you use the datasets available on the links above instead of directly from the Chicago Data Portal. The versions linked here are subsets of the original datasets and have some of the column names modified to be more database friendly which will make it easier to work out the problems.</p>

## Store the datasets in database tables

<p>To analyze the data using SQL, it first needs to be loaded into SQLite DB.
We will create three tables in as under:</p>
<ol>
    <li><b>CENSUS_DATA</b></li>
    <li><b>CHICAGO_PUBLIC_SCHOOLS</b></li>
    <li><b>CHICAGO_CRIME_DATA</b></li>
</ol>

<p>Load the <code>pandas</code> and <code>sqlite3</code> libraries and establish a connection to <code>FinalDB.db</code>.</p>

In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
dir_path = os.path.join(".", "data")
db_path = os.path.join(dir_path, "FinalDB.db")
os.makedirs(dir_path, exist_ok=True)

In [3]:
con = sqlite3.connect(db_path)
cur = con.cursor()

<p>Use <code>Pandas</code> to load the data available in the links above to dataframes. Use these dataframes to load data on to the database <code>FinalDB.db</code> as required tables.</p>

In [4]:
def load_data_to_db(url: str, table: str, connection: sqlite3.Connection, chunk_size: int | None=None) -> int | None:
    df = pd.read_csv(url)
    return df.to_sql(table, connection, if_exists="replace", index=False, method="multi", chunksize=chunk_size)

In [5]:
url_census_data = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv"
load_data_to_db(url_census_data, "chicago_census_data", con)

78

In [6]:
url_public_schools = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv"
load_data_to_db(url_public_schools, table="chicago_public_schools", connection=con, chunk_size=100)

566

In [7]:
url_crime_data = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv"
load_data_to_db(url_crime_data, table="chicago_crime_data", connection=con, chunk_size=100)

533

<p>You can now proceed to the following problems.</p>

## Problems

In [8]:
def sql_query_and_display(sql:str, cursor: sqlite3.Cursor) -> None:
    # get query results
    cursor.execute(sql)
    results = cursor.fetchall()

    # table header
    import html
    columns = [html.escape(desc[0]) for desc in cursor.description]

    # process data
    processed_rows = []
    for row in results:
        processed = []
        for val in row:
            if val is None:
                processed.append("None")
                continue

            processed.append(html.escape(str(val)))

        processed_rows.append(processed)

    # use html to display results
    html_table = f"""
    <table border="1" cellpadding="5" style="border-collapse: collapse; font-family: monospace;">
        <tr>{''.join(f'<th style="text-align: left">{col}</th>' for col in columns)}</tr>
        {''.join(f"<tr>{''.join(f'<td style="text-align: left">{cell}</td>' for cell in row)}</tr>" for row in processed_rows)}
    </table>
    """

    from IPython.display import display, HTML
    display(HTML(html_table))

<p><b>Problem 1</b> Find the total number of crimes recorded in the <code>crime</code> table.</p>

In [9]:
statement = "SELECT COUNT(*) FROM chicago_crime_data;"
sql_query_and_display(statement, cur)

COUNT(*)
533


<p><b>Problem 2</b> List community area names and numbers with per capita income less than 11000.</p>

In [10]:
statement = """
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME
FROM chicago_census_data
WHERE PER_CAPITA_INCOME < 11000;
"""
sql_query_and_display(statement, cur)

COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
26.0,West Garfield Park
30.0,South Lawndale
37.0,Fuller Park
54.0,Riverdale


<p><b>Problem 3</b> List all case numbers for crimes involving minors. (Children are not considered minors for the purposes of crime analysis)</p>

In [11]:
statement = """
SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION
FROM chicago_crime_data
WHERE UPPER(PRIMARY_TYPE) LIKE '%MINOR%' OR UPPER(DESCRIPTION) LIKE '%MINOR%';
"""
sql_query_and_display(statement, cur)

CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HL266884,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR


<p><b>Problem 4</b> List all kidnapping crimes involving a child.</p>

In [12]:
statement = """
SELECT *
FROM chicago_crime_data
WHERE UPPER(PRIMARY_TYPE) = 'KIDNAPPING' AND UPPER(DESCRIPTION) LIKE '%CHILD%';
"""
sql_query_and_display(statement, cur)

ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


<p><b>Problem 5</b> List the kind of crimes that were recorded at schools. (No repetitions)</p>

In [13]:
statement = """
SELECT DISTINCT PRIMARY_TYPE
FROM chicago_crime_data
WHERE UPPER(LOCATION_DESCRIPTION) LIKE '%SCHOOL%';
"""
sql_query_and_display(statement, cur)

PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


<p><b>Problem 6</b> List the type of schools along with the average safety score for each type.</p>

In [14]:
statement = """
SELECT
    `Elementary, Middle, or High School` AS SCHOOL_TYPE,
    AVG(`SAFETY_SCORE`) AS AVERAGE_SAFETY_SCORE
FROM chicago_public_schools
GROUP BY SCHOOL_TYPE;
"""
sql_query_and_display(statement, cur)

SCHOOL_TYPE,AVERAGE_SAFETY_SCORE
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


<p><b>Problem 7</b> List 5 community areas with highest percentage of households below proverty line.</p>

In [15]:
statement = """
SELECT
    COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM chicago_census_data
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC NULLS LAST LIMIT 5;
"""
sql_query_and_display(statement, cur)

COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
54.0,Riverdale,56.5
37.0,Fuller Park,51.2
68.0,Englewood,46.6
29.0,North Lawndale,43.1
27.0,East Garfield Park,42.4


<p><b>Problem 8</b> Which community area is most crime prone? Display the community area number only.</p>

In [16]:
statement = """
SELECT COMMUNITY_AREA_NUMBER
FROM chicago_crime_data
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(*) DESC LIMIT 1;
"""
sql_query_and_display(statement, cur)

COMMUNITY_AREA_NUMBER
25.0


<p><b>Problem 9</b> Use a sub-query to find the name of the community area with highest hardship index.</p>

In [17]:
statement = """
SELECT COMMUNITY_AREA_NAME, HARDSHIP_INDEX
FROM chicago_census_data
WHERE HARDSHIP_INDEX IN (
    SELECT MAX(HARDSHIP_INDEX)
    FROM chicago_census_data
);
"""
sql_query_and_display(statement, cur)

COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


<p><b>Problem 10</b> Use a sub-query to determine the community area name with most number of crimes.</p>

In [18]:
statement = """
SELECT COMMUNITY_AREA_NAME
FROM chicago_census_data
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM chicago_crime_data
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC LIMIT 1
);
"""
sql_query_and_display(statement, cur)

COMMUNITY_AREA_NAME
Austin


In [19]:
con.close()

****
This is the end of the file.
****